In [92]:
import math
import cv2
import numpy as np
import time
import random as rand
from operator import add, mul, sub

In [48]:
def is_border(canvas, height, width, c, r):
    return c == 0 or c == width - 1 or r == 0 or r == height - 1 or canvas[r - 1][c - 1] == 0 or canvas[r - 1][c] == 0 or canvas[r - 1][c + 1] == 0 or canvas[r][c - 1] == 0 or canvas[r][c + 1] == 0 or canvas[r + 1][c - 1] == 0 or canvas[r + 1][c] == 0 or canvas[r + 1][c + 1] == 0

def text_to_points(canvas):
    points = []
    height, width = canvas.shape
    for r in range(height):
        for c in range(width):
            if canvas[r][c] > 0 and is_border(canvas, height, width, c, r):
                points.append((c, r))
    return points

In [115]:
class Vehicle:
    def __init__(self, x, y):
        self.pos = [x, y]
        self.target = [x, y]
        self.vel = [rand.uniform(-1, 1), rand.uniform(-1, 1)]
        self.acc = [0, 0]
        self.r = 8
        self.max_speed = 4
        
    def update(self):
        self.pos = list(map(add, self.pos, self.vel))
        self.vel = list(map(add, self.vel, self.acc))
        self.acc = [0, 0]
        
    def draw(self, canvas):
        height, width = canvas.shape
        if self.pos[0] >= 0 and self.pos[0] < width and self.pos[1] >= 0 and self.pos[1] < height:
            canvas[int(self.pos[1]), int(self.pos[0])] = 255.0
            
    def behaviors(self):
        arrive = self.arrive(self.target)
        self.apply_force(arrive)
        
    def apply_force(self, seek):
        self.acc = list(map(add, self.acc, seek))
        
    def arrive(self, target):
        raw = list(map(sub, target, self.pos))
        mag = math.sqrt(raw[0]**2 + raw[1]**2)
        speed = self.max_speed
        if mag < 5:
            speed = mag / 5 * self.max_speed
        scale = speed / mag
        for i, val in enumerate(raw):
            raw[i] *= scale
        steer = list(map(sub, raw, self.vel))
        return steer        
        
    def seek(self, target):
        if target == self.pos:
            return [0, 0]
        
        raw = list(map(sub, target, self.pos))
        mag = math.sqrt(raw[0]**2 + raw[1]**2)
        scale = self.max_speed / mag
        for i, val in enumerate(raw):
            raw[i] *= scale
        steer = list(map(sub, raw, self.vel))
        return steer


In [116]:
height, width = (300, 600)
canvas = np.zeros((height, width))
cv2.putText(canvas, 'train', (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 0, 0), 12)

points = text_to_points(canvas)
vehicles = []
for point in points:
    vehicle = Vehicle(point[0], point[1])
    vehicles.append(vehicle)
    
while True:
    canvas = np.zeros((height, width))
    for i, vehicle in enumerate(vehicles):
        vehicle.behaviors()
        vehicle.update()
        vehicle.draw(canvas)
        
    cv2.imshow('canvas', canvas)
    k = cv2.waitKey(300) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()

ZeroDivisionError: float division by zero